# TEST REPORT

In [ ]:
import boto3

from aws.utils.notebooks.database import get_redshift
from aws.utils.notebooks.common import get_workspace,get_scratch_database
import os
import sys
import boto3
import aws.utils.notebooks.controller as controller
s3 = boto3.client('s3')
workspace = get_workspace()
team_space = workspace['team_space']
notebook_bucket = workspace['NotebookOutputBucket']
workspace

### Configure Result Report

In [ ]:
notebooks_result_config = {
    "exclude_list":  ["Z-Tests/Test-Report/"],       
    "optional_list": ["B-DataAnalyst/Example-3-PySpark-SQL",
                      "Example-4-SQL-Analysis-Spark",
                      "B-DataAnalyst/Example-7-Data-Profiling",
                      "J-FHIR-Development/Example-0-Build-Schema-Induction",
                      "J-FHIR-Development/Example-1-CreateTables",
                      "J-FHIR-Development/Example-2-FHIR-Queries"],
    "minimum_successful": 2    # number of minimum notebooks to be completed to consider entire test not failed
}

def checkPath(path):
    for p in notebooks_result_config["exclude_list"]:
        if p in path:
            return "ignore"
    
    ignored = False
    for p in notebooks_result_config["optional_list"]:
        if p in path:
            ignored = True
    
    nb_name = path.split("/")[-2] # Name of the folder for the output file is a notebook name
    res = path.split("/")[-1] # result of notebook actual execution (output)
    if 'error@' in res and 'Failure-Behavior' not in nb_name:
        return "failed" if not ignored else "ignore"
    elif 'error@' in res and 'Failure-Behavior' in nb_name:
        return "success"
    elif 'error@' not in res and 'Failure-Behavior' in nb_name:
        return "failed" if not ignored else "ignore"
    elif 'error@' not in res and 'Failure-Behavior' not in nb_name:
        return "success"
    else:
        return "ignore" #impossible to be here.


In [ ]:

notebooks_pass = []
notebooks_error = []
notebooks_ignored = []

for key in s3.list_objects_v2(Bucket=notebook_bucket, Prefix=f'{team_space}/output/notebooks/tests/')['Contents']:
    path = key['Key']
    execution_result = checkPath(path)
    if execution_result == "ignore":
        notebooks_ignored.append(path)
    elif execution_result == "failed":
        notebooks_error.append(path)
    elif execution_result == "success":
        notebooks_pass.append(path)

print("Notebooks ignored:\n")
display(notebooks_ignored)
print("\n------------------------")
print("Notebooks with errors:\n")
display(notebooks_error)
print("\n------------------------")
print("Notebooks passed: \n") 
display(notebooks_pass)

assert not notebooks_error, 'Some notebooks have failed'
assert len(notebooks_pass) >= notebooks_result_config["minimum_successful"], \
    f'Total passed {len(notebooks_pass)} is less than configured minimum {notebooks_result_config["minimum_successful"]}'
